# RSSI Experiement
## Test RSSI by distance (0cm ~ 150cm)
### Date: 05/10/2022
### Goal: Mean, Median, Mod (iOS, AOS)

iBeacon과 iPhone12를 활용하여 RSSI값을 거리별로 측정한다. 거리 측정을 활용해서 RSSI 값의 분포를 알아낸다. 실험의 목적은 이후에 우리가 RSSI 값을 비콘으로 수신받았을때, 어느정도 위치에 존재하는지 알 수 있도록 하는 것이다. Indoor Localization의 기초 실험

### 실험 논의
간격에 대한 논의가 이루어졌다. 10cm 단위로는 판별을 할 수 없다! 그래서 새로운 실험계획이 필요

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode

In [53]:
sourceAddress = '/Users/jungpeter/Desktop/Develop/project/BEST/iOS_RSSI_Distance_Experiment_CSV/exp1/'
aosSourceAddress = '/Users/jungpeter/Desktop/Develop/project/BEST/AOS_RSSI_Distance_Experiment_CSV/exp1/'

ios_dfs = []
aos_dfs = []
dist = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]

ios_rssi_stat = {'dist': dist, 'mean': [], 'median': [], 'mod': []}
aos_rssi_stat = {'dist': dist, 'mean': [], 'median': [], 'mod': []}
ios_rssi = {}
aos_rssi = {}
data_limit = 250


for i in range(0, 16):
    ios_dfs.append(pd.read_csv(sourceAddress+f'iOS_exp1_{i*10}.csv'))
    aos_dfs.append(pd.read_csv(aosSourceAddress+f'AOS_exp1_{i*10}.csv'))

In [54]:
for df in ios_dfs:
    newTS = []
    for timestamp in df["timeStamp"]:
#         newTS.append(" ".join(timestamp.split(' ')[1:2]))
        newTS.append(timestamp)
    total_len = len(newTS)
    left = total_len // 2
    right = total_len // 2 + 1
    result = []
    
    # 250 개가 되지 않을때
    if total_len < data_limit:
        rssiArr = []
        for rssi in df['RSSI']:
            rssiArr.append(rssi)
        ios_rssi_stat['mean'].append(np.mean(rssiArr))
        ios_rssi_stat['median'].append(np.median(rssiArr))
        ios_rssi_stat['mod'].append(int(mode(rssiArr)[0]))
    # 250개 이상일때
    else:
        while(True):
            if len(result) >= data_limit:
                break
            if left < 0 and right > total_len:
                break
            if left >= 0:
                result.insert(0, newTS[left])
                left = left - 1
            if right < total_len:
                result.append(newTS[right])
                right = right + 1

        first = df[df['timeStamp'] == result[0]].index.values
        last = df[df['timeStamp'] == result[-1]].index.values

        rssiArr = []
        for rssi in df[first[0]:last[0]+1]['rssi']:
            rssiArr.append(rssi)
        ios_rssi_stat['mean'].append(np.mean(rssiArr))
        ios_rssi_stat['median'].append(np.median(rssiArr))
        ios_rssi_stat['mod'].append(int(mode(rssiArr)[0]))

In [55]:
for df in aos_dfs:
    newTS = []
    for timestamp in df["Date"]:
        newTS.append(timestamp)
    total_len = len(newTS)
    left = total_len // 2
    right = total_len // 2 + 1
    result = []
    
    #250개 이하일때
    if total_len < data_limit:
        rssiArr = []
        for rssi in df['RSSI']:
            rssiArr.append(rssi)
            
        aos_rssi_stat['mean'].append(np.around(np.mean(rssiArr), 3))
        aos_rssi_stat['median'].append(np.around(np.median(rssiArr), 3))
        aos_rssi_stat['mod'].append(np.around(int(mode(rssiArr)[0]), 3))
        
    # 250개 이상일때 250개 짜르기
    else:
        while(len(result) < data_limit):
            if left < 0 and right > total_len:
                break
            if left >= 0:
                result.insert(0, newTS[left])
                left = left - 1
            if right < total_len:
                result.append(newTS[right])
                right = right + 1

        first = df[df['Date'] == result[0]].index.values
        last = df[df['Date'] == result[-1]].index.values

        rssiArr = []
        for rssi in df[first[0]:last[0]+1]['RSSI']:
            rssiArr.append(rssi)
            
        aos_rssi_stat['mean'].append(np.around(np.mean(rssiArr), 3))
        aos_rssi_stat['median'].append(np.around(np.median(rssiArr), 3))
        aos_rssi_stat['mod'].append(np.around(int(mode(rssiArr)[0]), 3))

In [56]:
pd.DataFrame(ios_rssi_stat)

,dist,mean,median,mod
0,0,-21.888,-21.0,-21
1,10,-34.548,-35.0,-36
2,20,-33.212,-33.0,-34
3,30,-48.224,-49.0,-50
4,40,-53.932,-53.0,-53
5,50,-51.500,-52.0,-52
6,60,-47.828,-48.0,-49
7,70,-45.068,-45.0,-45
8,80,-46.156,-46.0,-46
9,90,-48.860,-49.0,-48


In [57]:
pd.DataFrame(aos_rssi_stat)

,dist,mean,median,mod
0,0,-26.420,-26.0,-28
1,10,-37.258,-37.0,-36
2,20,-44.464,-43.0,-42
3,30,-41.128,-42.0,-42
4,40,-45.976,-46.0,-47
5,50,-50.349,-50.0,-51
6,60,-49.164,-49.0,-47
7,70,-47.856,-48.0,-48
8,80,-49.608,-50.0,-51
9,90,-48.180,-48.0,-48
